In [1]:
import csv
import math
import re
import pandas as pd
import requests
import numpy as np
import io

# Collecting the data

The Dataset cannot be pushed to Github. Github has decreased the allowed file size so the dataset will have to be downloaded manually.

The url is: https://data.cms.gov/provider-data/sites/default/files/resources/69a75aa9d3dc1aed6b881725cf0ddc12_1671768321/DAC_NationalDownloadableFile.csv

The Data Dictionary for this section of datasets: https://data.cms.gov/provider-data/sites/default/files/data_dictionaries/physician/DOC_Data_Dictionary.pdf

The webpage for this section of datasets: https://data.cms.gov/provider-data/topics/doctors-clinicians


In [2]:
df = pd.read_csv('NationalDocData/DAC_NationalDownloadableFile.csv', encoding='windows-1252', dtype='string')

(TODO) Filtering and Cleaning the Dataset

Note: Since this is for Physicians, we will need to filter out non-physicians, but keep in mind that some physicians will also have a secondary degree listed under their specialities, so the filtering needs to be a bit smarter

In [3]:
# NOTE: Optimize This
addresses = df[['adr_ln_1', 'cty', 'st', 'zip','pri_spec']]
addresses.drop_duplicates()
addresses.dropna()
addresses = addresses[addresses['adr_ln_1'].str.strip().astype(bool)]
addresses = addresses[addresses['cty'].str.strip().astype(bool)]
addresses = addresses[addresses['st'].str.strip().astype(bool)]
addresses = addresses[addresses['zip'].str.strip().astype(bool)]
addresses = addresses[addresses['pri_spec'].str.strip().astype(bool)]
addresses['pri_spec'] = addresses['pri_spec'].astype(str)
print(addresses.head())
print(addresses.shape[0])

             adr_ln_1             cty  st        zip                pri_spec
0     1582 N BROAD ST        TAZEWELL  TN  378794352      NURSE PRACTITIONER
1   5632 ANNAPOLIS RD     BLADENSBURG  MD  207102213      NURSE PRACTITIONER
2    611 E CARLSON ST        CHEYENNE  WY  820094335       INTERNAL MEDICINE
3         30 RIVER ST      JIM THORPE  PA  182292313            CHIROPRACTIC
4  20501 VENTURA BLVD  WOODLAND HILLS  CA  913640847  CLINICAL SOCIAL WORKER
2472219


In [4]:
#Printing Unique Primary Specialites to Filter From
print(addresses['pri_spec'].unique())

['NURSE PRACTITIONER' 'INTERNAL MEDICINE' 'CHIROPRACTIC'
 'CLINICAL SOCIAL WORKER' 'OPTOMETRY' 'ANESTHESIOLOGY' 'FAMILY PRACTICE'
 'NEUROLOGY' 'CLINICAL PSYCHOLOGIST' 'DERMATOLOGY' 'PHYSICAL THERAPY'
 'OPHTHALMOLOGY' 'ORTHOPEDIC SURGERY' 'HOSPITALIST' 'PSYCHIATRY'
 'ORAL SURGERY' 'GENERAL PRACTICE' 'PLASTIC AND RECONSTRUCTIVE SURGERY'
 'PHYSICAL MEDICINE AND REHABILITATION' 'GASTROENTEROLOGY' 'UROLOGY'
 'PULMONARY DISEASE' 'CERTIFIED REGISTERED NURSE ANESTHETIST (CRNA)'
 'REGISTERED DIETITIAN OR NUTRITION PROFESSIONAL' 'PODIATRY'
 'ENDOCRINOLOGY' 'CARDIOVASCULAR DISEASE (CARDIOLOGY)'
 'MAXILLOFACIAL SURGERY' 'QUALIFIED AUDIOLOGIST' 'OBSTETRICS/GYNECOLOGY'
 'PHYSICIAN ASSISTANT' 'PEDIATRIC MEDICINE' 'NEUROSURGERY' 'RHEUMATOLOGY'
 'QUALIFIED SPEECH LANGUAGE PATHOLOGIST' 'INTERVENTIONAL PAIN MANAGEMENT'
 'OCCUPATIONAL THERAPY' 'EMERGENCY MEDICINE' 'PAIN MANAGEMENT'
 'INTERVENTIONAL CARDIOLOGY' 'NEPHROLOGY' 'INFECTIOUS DISEASE'
 'GENERAL SURGERY' 'GERIATRIC MEDICINE' 'ALLERGY/IMMUNOLOGY'
 

In [5]:
batches = math.ceil(addresses.shape[0] / 10000)
print(batches)

248


# Getting Geolocation

Create a loop that generates a CSV for each 247 entries
Send the generated CSV to the Census GOV Geolocation API for processing
Recieve the response and append it to a local CSV
Rinse and repeat until all addresses are converted to geolocations (lat. and long.) to use with a map

In [6]:
URL = 'https://geocoding.geo.census.gov/geocoder/locations/addressbatch'
with open('NationalDocData/test_addresses.csv', 'rb') as input_file:
    geo = requests.post(files={'addressFile': input_file}, url=URL, data={'benchmark':'Public_AR_Current'})

In [7]:
print(geo.content)

b'"000001","20 W 34th St., New York,  NY,  10001","Match","Non_Exact","20 W 34TH ST, NEW YORK, NY, 10118","-73.98533698799997,40.748757279000074","59653429","L"\n'


In [8]:
print(type(geo))

<class 'requests.models.Response'>


In [9]:
#data = need to change response to text and separate values

df = pd.read_csv(io.StringIO(geo.text), sep=',', header=None, quoting=csv.QUOTE_ALL, index_col= False)

print(df)

   0                                     1      2          3  \
0  1  20 W 34th St., New York,  NY,  10001  Match  Non_Exact   

                                   4                                      5  \
0  20 W 34TH ST, NEW YORK, NY, 10118  -73.98533698799997,40.748757279000074   

          6  7  
0  59653429  L  


In [10]:
#find latitude and longitude column
df.reset_index()

,index,0,1,2,3,4,5,6,7
0,0,1,"20 W 34th St., New York, NY, 10001",Match,Non_Exact,"20 W 34TH ST, NEW YORK, NY, 10118","-73.98533698799997,40.748757279000074",59653429,L


If you have a DataFrame with only one row, then access the first (only) row as a Series using iloc, and then the value using the column name:

In [11]:
coord = df.iloc[0][5]
lat, lon = coord.split(',')

In [12]:
lat

'-73.98533698799997'

In [13]:
lon

'40.748757279000074'

In [14]:
print(type(lon))

<class 'str'>


In [15]:
print(df[1].to_string(index=False))

20 W 34th St., New York,  NY,  10001


# Creating dictionary to create dataframe (preparing csv)
- create dictionary
- pull data from response (geo) and previously made dataframe (addresses)
- create method to save into csv (probably not necessary at this step, but wanted to figure out a method when exporting)

In [17]:
new_dict = {}
new_dict['Address'] = df[1].to_string(index=False)
new_dict['Latitude'] = lat
new_dict['Longitude'] = lon
primary= addresses['pri_spec'].iloc[0]
new_dict['Primary'] = primary

print(new_dict)

{'Address': '20 W 34th St., New York,  NY,  10001', 'Latitude': '-73.98533698799997', 'Longitude': '40.748757279000074', 'Primary': 'NURSE PRACTITIONER'}


In [18]:
df = pd.DataFrame(new_dict, index=[0])
print(df)

                                Address            Latitude  \
0  20 W 34th St., New York,  NY,  10001  -73.98533698799997   

            Longitude             Primary  
0  40.748757279000074  NURSE PRACTITIONER  


In [22]:
from pathlib import Path

output_file = 'my_file.csv'
output_dir = Path('Users/johannahoang/PycharmProjects/NationalDocData')

output_dir.mkdir(parents=True, exist_ok=True)

df.to_csv(output_dir / output_file)  # can join path elements with / operator
print(df)

                                Address            Latitude  \
0  20 W 34th St., New York,  NY,  10001  -73.98533698799997   

            Longitude             Primary  
0  40.748757279000074  NURSE PRACTITIONER  


In [ ]:
# data = addresses[['Latitude', 'Longitude', 'Specialty']]
# df2 = pd.DataFrame([[lat,lon,addresses['pri_spec'].iloc[0]]], columns=['Latitude','Longitude', 'Specialty'])
# pd.concat([df2, data])
# print(data)
# # xdf = xdf.reindex(columns = header_list)  
# header_list = 
# xdf.loc[-1] = [2, 3, 4]  # adding a row
# xdf.index = df.index + 1  # shifting index
# xdf = df.sort_index()  # sorting by index


In [ ]:
TODO: figure out how to batch size loop

# batch_size = 
# data = [1, 1, 2, 3, 5, 8, 13, ...]

# for i in range(0, len(data), batch_size):
#     print(data[i:i+batch_size])